In [1]:
import __future__

import numpy as np

from qiskit.quantum_info import SparsePauliOp

## Exercise

In the tutorial, we implemented the Jordan-Wigner transformation for a 2-body hopping term $a_p^\dagger a_q$ for the case $p > q$. However, in general, this is not the only case a mapper has to cover. You should implement the missing cases for 2-body terms, namely
- $a_p^\dagger a_q$ with $p < q$
- The transpose cases $a_p a_q^\dagger$ with $p > q$ and $p < q$
- The number operator $n = a_p^\dagger a_p$ ($p = q$)

The class below stores s **second quantization operator** as strings in the following format: \
$a_p^\dagger$ --> `"+_p"` \
$a_q$ --> `"-_q"` \
where the first character (+/-) indicates whether it is a creation or annihilation operator, and the last character specifies the index of the orbital/qubit it is acting on. \
Multiple operators are always separated by a space. Two-body terms, for example, are encoded as \
$a_p^\dagger a_q$ --> `"+_p -_q"`

## Solution

In [25]:
class FermionicOperator():

    def __init__(self, op_list: list | np.ndarray, num_qubits: int=None):
        self.op_list = op_list
        self.num_qubits = num_qubits

        self.ops_split = []

        max_idx = 0
        for op, coeff in self.op_list.items():
            op_split = op.split()
            num_ops = len(op_split)
            self.ops_split.append({
                "indices": [int(o[-1]) for o in op_split],
                "conjugation": [o[0] for o in op_split],
                "coeff": coeff
            })

            if max_idx < max(self.ops_split[-1]["indices"]):
                max_idx = max(self.ops_split[-1]["indices"])

            if num_ops % 2 != 0 or self.ops_split[-1]["conjugation"].count("+") != num_ops / 2:
                raise ValueError(f"FermionicOperator.map_operator(): {op} invalid")

        if self.num_qubits is None:
            self.num_qubits = max_idx + 1
        elif self.num_qubits < max_idx + 1:
            raise ValueError(
                f"FermionicOperator.__init__(): num_qubits ({self.num_qubits}) "
                f"is smaller than maximum operator index")

    def map_operator(self):
        mapped_op_list = []
        for i, op in enumerate(self.ops_split):
            if len(op["indices"]) == 2 and op["indices"][0] == op["indices"][1]:
                mapped_op_list.append(self._map_number_op(op))
            elif len(op["indices"]) == 2 and op["indices"][0] != op["indices"][1]:
                mapped_op_list.append(self._map_two_body_hop_op(op))
            
        return mapped_op_list

    def _map_number_op(self, op):

        return None
    
    def _map_two_body_hop_op(self, op):
        # p > q:
        # (X - iY)_p Z_p-1 ... Z_q+1 (X + iY)_q

        mapped_op = []
        norm = 1./4

        idx0 = op["indices"][0]
        idx1 = op["indices"][1]

        mapped_op.append((
            (self.num_qubits - idx0 - 1) * "I" + "X" + (idx0 - idx1 - 1) * "Z" + "X" + idx1 * "I",
            op["coeff"] * norm
        ))
        mapped_op.append((
            (self.num_qubits - idx0 - 1) * "I" + "Y" + (idx0 - idx1 - 1) * "Z" + "X" + idx1 * "I",
            - 1.j * op["coeff"] * norm
        ))
        mapped_op.append((
            (self.num_qubits - idx0 - 1) * "I" + "X" + (idx0 - idx1 - 1) * "Z" + "Y" + idx1 * "I",
            1.j * op["coeff"] * norm
        ))
        mapped_op.append((
            (self.num_qubits - idx0 - 1) * "I" + "Y" + (idx0 - idx1 - 1) * "Z" + "Y" + idx1 * "I",
            op["coeff"] * norm
        ))

        return SparsePauliOp.from_list(mapped_op)


In [26]:
# test cases:
op_list = [
    # ({second_q-operator: coefficient}, num_qubits)
    ({"+_2 -_0": 1.0}, 3),  # a^dagger_2 a_0
    ({"-_1 +_0": 1.0}, 3),  # a_1 a^dagger_0
    ({"+_0 -_2": 1.0}, 3),  # a^dagger_0 a_2
    ({"-_2 +_1": 1.0}, 3),  # a_2 a^dagger_1
    ({"+_1 -_2": 1.0}, 3),  # a^dagger_1 a_2
    ({"-_1 +_2": 1.0}, 3),  # a_1 a^dagger_2
    ({"+_1 -_1": 1.0}, 3),  # a^dagger_1 a_1
]

op_mapped = []
for op in op_list:
    ferm = FermionicOperator(op[0], num_qubits=op[1])
    op_mapped.append(ferm.map_operator()[0])
    print(op_mapped[-1])

SparsePauliOp(['XZX', 'YZX', 'XZY', 'YZY'],
              coeffs=[0.25+0.j  , 0.  -0.25j, 0.  +0.25j, 0.25+0.j  ])
SparsePauliOp(['IXX', 'IYX', 'IXY', 'IYY'],
              coeffs=[0.25+0.j  , 0.  -0.25j, 0.  +0.25j, 0.25+0.j  ])
SparsePauliOp(['IIXXII', 'IIYXII', 'IIXYII', 'IIYYII'],
              coeffs=[0.25+0.j  , 0.  -0.25j, 0.  +0.25j, 0.25+0.j  ])
SparsePauliOp(['XXI', 'YXI', 'XYI', 'YYI'],
              coeffs=[0.25+0.j  , 0.  -0.25j, 0.  +0.25j, 0.25+0.j  ])
SparsePauliOp(['IXXII', 'IYXII', 'IXYII', 'IYYII'],
              coeffs=[0.25+0.j  , 0.  -0.25j, 0.  +0.25j, 0.25+0.j  ])
SparsePauliOp(['IXXII', 'IYXII', 'IXYII', 'IYYII'],
              coeffs=[0.25+0.j  , 0.  -0.25j, 0.  +0.25j, 0.25+0.j  ])
None


In [27]:
test_ops = {
    "+_2 -_0": SparsePauliOp(
        ["XZY", "XZX", "YZY", "YZX"],
        coeffs=[0.0 + 0.25j, 0.25 + 0.0j, 0.25 + 0.0j, 0.0 - 0.25j],
    ),
    "-_1 +_0": SparsePauliOp(
        ["IXY", "IXX", "IYY", "IYX"],
        coeffs=[0.0 + 0.25j, -0.25 + 0.0j, -0.25 + 0.0j, 0.0 - 0.25j],
    ),
    "+_0 -_2": SparsePauliOp(
        ["XZY", "YZY", "XZX", "YZX"],
        coeffs=[0.0 - 0.25j, 0.25 + 0.0j, 0.25 + 0.0j, 0.0 + 0.25j],
    ),
    "-_2 +_1": SparsePauliOp(
        ["XYI", "XXI", "YYI", "YXI"],
        coeffs=[0.0 + 0.25j, -0.25 + 0.0j, -0.25 + 0.0j, 0.0 - 0.25j],
    ),
    "+_1 -_2": SparsePauliOp(
        ["XYI", "YYI", "XXI", "YXI"],
        coeffs=[0.0 - 0.25j, 0.25 + 0.0j, 0.25 + 0.0j, 0.0 + 0.25j],
    ),
    "-_1 +_2": SparsePauliOp(
        ["XYI", "YYI", "XXI", "YXI"],
        coeffs=[0.0 - 0.25j, -0.25 + 0.0j, -0.25 + 0.0j, 0.0 + 0.25j],
    ),
    "+_1 -_1": SparsePauliOp(
        ["III", "IZI"],
        coeffs=[0.5 + 0.0j, -0.5 + 0.0j]
    ),
}

In [28]:
for i, k in enumerate(test_ops):
    print(f"{k} correctly mapped: ", end="")
    if op_mapped[i] is not None:
        print(op_mapped[i].sort() == test_ops[k].sort())
    else:
        print(False)

+_2 -_0 correctly mapped: True
-_1 +_0 correctly mapped: False
+_0 -_2 correctly mapped: False
-_2 +_1 correctly mapped: False
+_1 -_2 correctly mapped: False
-_1 +_2 correctly mapped: False
+_1 -_1 correctly mapped: False
